# Exploring OpenBible.info Data

This dataset by Stephen Smith has a collection of topics and associated verses, and was initiated in 2007. The initial set was about 4000 topics based on completing the phrase "What does the Bible say about..."). After that:

> I used the Yahoo Web Search API to get the top thirty webpages related to each topic and then extracted the verse references from each page.

(This approach likely included some irrelevant verses: in theory the voting approach should mitigate this.)

> This Bible is a mashup of the Yahoo! and ESV Bible web services. It searches the Internet for the topics that interest people, many of which you’d never find in a traditional topical Bible. Then it shows relevant verses.

Because of the methodology for adding topics, this data might be most useful if combined with other, more curated topic inventories.

On the resulting site, users are invited to 
- vote on the relevance of the verse to the topics
- suggest other verses for a topic
- suggest new topics

> Since launching three weeks ago, people have voted up or down 3,000 verses and suggested 200 new verses, in addition to creating 500 new topics.

The original site had a passage -> tag cloud feature that no longer appears to function.

## Scoring

The initial scores:

> each page got one vote per unique verse—so two references to John 1:1 on the same page would only count as one vote. All verses that appeared on two or more webpages made it into the main TB index.

(This suggests that any topic/passage pair which still has only two votes might be discardable.)

> About 750 of the topics occurred in both the new TB and in Nave’s; every verse for each topic in Nave’s got an extra three votes in the new TB.

* Initial release: [June « 2007 « OpenBible.info Blog](https://www.openbible.info/blog/2007/06/)
* [Topical Bible Technical Notes « OpenBible.info Blog](https://www.openbible.info/blog/2007/07/topical-bible-technical-notes/)
* Other blog posts on the category Topic: https://www.openbible.info/blog/category/topics/. This includes some interesting change-over-time analysis for some hot-button topics.

## Duplicate Topics

> Searching for a word will automatically add it.

This means there's some duplication that should probably be collapsed, e.g. 

```
Tatoos On The Body
...
Tattoo
Tattooing
Tattooing Your Body
Tattoos
Tattoos And Body Piercings
Tattoos And Piercings
Tattoos Body Piercings
```

## Updates

Note this data is still updated weekly: this snapshot is from 2024-08-05. It might be interesting to compare against previous versions to see whether the data is still growing, and how. For example, the letter T currently has 734 entries: it had the same number of entries in 2014.

The analysis below shows:
* There are now 6700 unique topics. So clearly some de-duping and consolidation may be needed.
* There is a "fat head" of topics with a vote count way above the mean

In [1]:
# Setup
import pandas as pd
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

from src.openbibleinfo import DATAPATH, reader
rd = reader.Reader()

In [2]:
# some passages are ranges (EXO 20:1-26), others a single verse (Gal 5:14)
rd.df.head()

,Topic,StartVerseId,EndVerseId,Votes,PassageLength,UsableRange,UsableReference
0,10 commandments,02020001,02020026,291,26,True,True
1,10 commandments,48005014,<NA>,140,1,False,True
2,10 commandments,45013008,45013010,114,3,True,True
3,10 commandments,05004013,<NA>,101,1,False,True
4,10 commandments,02034028,<NA>,93,1,False,True


In [3]:
n_topics = len(rd.df.Topic.value_counts())
n_records = len(rd.df)
n_votes = rd.df.Votes.sum()
print(f"Number of uniq topics: \t{n_topics:9}")
print(f"Number of rows: \t{n_records:9}")
print(f"Number of votes: \t{n_votes:9}")

Number of uniq topics: 	     6713
Number of rows: 	   465956
Number of votes: 	 73692178


## Expanding the Data: Passage Length

Some verses have an `EndVerseId` value, indicating a range. This adds a `PassageLength` column. 
* The value is 1 if no `EndVerseId`
* If the `EndVerseId` is in a different chapter, arbitrarily return length == 99. My code can't currently enumerate these verses, and potentially too long anyway.
* If the `EndVerseId` is in a different book, return 999. These ranges seem way too long to be useful. There are 72 of them, some of them hot button topics. Less than 1/1000 votes. 

`UsableRange` is True if a range and length < 99. `UsableReference` is True if a single verse or `UsableRange` is True. 

In [4]:
# cross-chapter ranges
ccrecords = rd.df[rd.df.PassageLength==99]
print(f"{len(ccrecords)} cross-chapter records")
print(f"{(ccrecords.Votes.sum() / n_votes)*100:.2f}% of all votes")
ccrecords.Topic.value_counts()

8373 cross-chapter records
0.95% of all votes


Topic
islam               131
being single        107
hate                101
abortion             96
sports               90
                   ... 
annoying people       1
ankh                  1
animal cruelty        1
anger management      1
666                   1
Name: count, Length: 1235, dtype: Int64

In [5]:
# cross-book ranges: these will likely get dropped
cbrecords = rd.df[rd.df.PassageLength==999]
print(f"{(cbrecords.Votes.sum() / n_votes)*100:.3f}% of all votes")
cbrecords.Topic.value_counts()

0.005% of all votes


Topic
islam                    7
being single             6
abortion                 6
christmas                5
video games              3
                        ..
violence                 1
watching tv              1
atheists                 1
losing your salvation    1
yoga                     1
Name: count, Length: 72, dtype: Int64

In [6]:
# topicsdf removes passages whose UsableRange < 99
n_topics = len(rd.topicsdf.Topic.value_counts())
n_records = len(rd.topicsdf)
n_votes = rd.topicsdf.Votes.sum()
print(f"Number of uniq topics: \t{n_topics:9}")
print(f"Number of rows: \t{n_records:9}")
print(f"Number of votes: \t{n_votes:9}")

Number of uniq topics: 	     6712
Number of rows: 	   457465
Number of votes: 	 72988030


### Distribution of Passage Lengths

This excludes cross-chapter and cross-book ranges. 

Looks like lengths <= 5 cover about 90% of the data. So perhaps enumerate and split votes up to that length, and otherwise treat as atomic (perhaps with a constant epsilon weight)? 

In [7]:
print(f"Records with a 'good' PassageLength value: {len(rd.topicsdf)} ({(len(rd.topicsdf)/n_records)*100:.2f}%)")
rd.topicsdf.PassageLength.value_counts(normalize=True)

Records with a 'good' PassageLength value: 457465 (100.00%)


PassageLength
1      0.723061
2      0.092969
3      0.042412
4      0.020555
5      0.013761
         ...   
107    0.000002
76     0.000002
70     0.000002
110    0.000002
79     0.000002
Name: proportion, Length: 80, dtype: float64

## Expanding the Data: aggregating intersecting references

Topic references sometimes intersect for a topic: for example "marrying a divorced woman" has votes for MAT 5:31, 5:32, and the range 5:31-32 (among others). Combining these would significantly increase the VotesPercentage for this passage.

In other cases there might be two ranges, one subsumed in the other (in this example, MAT 5:1-48 and MAT 5:21-48). Note the first reference (which has more votes) is the entire chapter. 

If we had a reasonable way to combine such references, that should sharpen the counts. Minimally, a single verse and a "short" range ought to be combinable: perhaps "short" here is PassageLength <= 3 or 5? 

The simplest initial step might be to combine single verses within a topic within an intersecting 2-verse range for that topic, allocating some portion of their votes: all? 0.75? 0.5? 

NOT YET ACCOMPLISHED.

In [28]:
marryingdivorced = rd.topicsdf[rd.topicsdf.Topic == "marrying a divorced woman"]
marryingdivorced[marryingdivorced.StartVerseId.str.startswith("40005")]

,Topic,StartVerseId,EndVerseId,Votes,PassageLength,UsableRange,UsableReference,VotesPercentage
263852,marrying a divorced woman,40005032,<NA>,6240,1,False,True,0.028619
263857,marrying a divorced woman,40005031,40005032,4931,2,True,True,0.022616
263885,marrying a divorced woman,40005031,<NA>,1862,1,False,True,0.008540
263925,marrying a divorced woman,40005001,40005048,1033,48,True,True,0.004738
263927,marrying a divorced woman,40005028,<NA>,982,1,False,True,0.004504
263971,marrying a divorced woman,40005002,<NA>,289,1,False,True,0.001325
263976,marrying a divorced woman,40005004,<NA>,273,1,False,True,0.001252
263987,marrying a divorced woman,40005001,<NA>,216,1,False,True,0.000991
264002,marrying a divorced woman,40005005,<NA>,183,1,False,True,0.000839
264015,marrying a divorced woman,40005003,<NA>,150,1,False,True,0.000688


## "Top" Topics

From here onward, we only consider records where `UsableReference` is True. So either a range within a chapter, or a single verse. 

A TopicRecord combines a Topic label, a set of passages, and a count of votes. 
* `TopicPassageCount` is the count of passages for a topic
* `TopicVotesSum` are the sum of votes for a topic
* `MeanPassageVotes` is `TopicVotesSum`/`TopicPassageCount`


In [8]:
# 55 is the 80% value for MeanPassageVotes across rd.topicsdf
# so an approximation for "high number of votes per passage"
#toptopics = rd.top_topics(threshold=55)
#toptopics.sort_values('MeanPassageVotes', ascending=False).head(10)
rd.toptopics.head(10)

,Topic,TopicPassageCount,TopicVotesSum,MeanPassageVotes
2785,helping others,592,829527,1401.228041
3027,immigration,364,430199,1181.865385
1812,eating pork,410,464953,1134.031707
5351,sodomy,476,478422,1005.088235
489,being born again,261,261267,1001.022989
4528,power of words,743,709295,954.636608
1338,covenant,322,294071,913.263975
2866,homosexuality,317,288815,911.088328
5316,slavery,539,484235,898.395176
2577,gossip,419,374752,894.396181


In [9]:
# the median TopicPassageCount value is 161: that still seems like way too many
rd.toptopics.describe()

,TopicPassageCount,TopicVotesSum,MeanPassageVotes
count,1378.000000,1378.000000,1378.000000
mean,185.872279,48137.674165,178.962464
std,131.987053,83964.933039,175.793107
min,1.000000,66.000000,55.028571
25%,107.000000,7504.250000,66.921060
50%,156.000000,15163.500000,96.593184
75%,236.000000,50480.250000,220.169807
max,1022.000000,829527.000000,1401.228041


In [10]:
# but not all "toptopics" seem "top"
rd.toptopics[rd.toptopics.TopicVotesSum < 1000].head()

,Topic,TopicPassageCount,TopicVotesSum,MeanPassageVotes
3285,kabzeel,4,657,164.250000
5257,shrimp,3,384,128.000000
2603,gray hair,8,974,121.750000
4110,nose piercings,6,677,112.833333
976,changing,1,112,112.000000


In [11]:
# top 50% by TopicPassageCount, looking at the bottom
rd.toptopics[rd.toptopics.TopicPassageCount >= 156].tail()

,Topic,TopicPassageCount,TopicVotesSum,MeanPassageVotes
1397,cursing,175,9672,55.268571
1263,confronting sins,173,9552,55.213873
4616,preventing conflict,189,10431,55.190476
3300,keeping promises,158,8718,55.177215
170,angels,189,10426,55.164021


In [12]:
# we might further filter rd.topicsdf to the top n passages for each topic in toptopics.Topic
# we could also choose a higher threshold for top_topics()
#rd.topicsdf[rd.topicsdf.Topic == "love one another"]
rd.topicsdf[rd.topicsdf.Topic == "immigration"]

,Topic,StartVerseId,EndVerseId,Votes,PassageLength,UsableRange,UsableReference
210840,immigration,03019033,03019034,19080,2,True,True
210841,immigration,02022021,<NA>,12943,1,False,True
210842,immigration,02023009,<NA>,10867,1,False,True
210843,immigration,40025035,<NA>,10510,1,False,True
210844,immigration,05027019,<NA>,10020,1,False,True
...,...,...,...,...,...,...,...
211202,immigration,51004001,51004018,10,18,True,True
211203,immigration,24029011,<NA>,10,1,False,True
211204,immigration,05023002,<NA>,10,1,False,True
211205,immigration,03024016,03024022,10,7,True,True


### Top Passages for Top Topics

For rd.toptopics collect the verses whose 
`n_votes` is the total number of votes. 

### Quality Score

Stephen measured quality in topic-scores.txt as "percentage of votes for the passage". This is done here with `passage_probability()`, only computed for toptopics. 

In [13]:
# compute across all topics: this takes a few minutes
#rd.topicsdf["VotesPercentage"] = rd.topicsdf.apply(rd.passage_probability, axis=1)
# this approach seems equally slow and produces the same warningb
#rd.topicsdf["VotesPercentage"] = rd.topicsdf.apply(rd.passage_probability2, axis=1)
# not sure how to address the warning this produces
# this approach seems faster: still produces the SettingWithCopyWarning warning though
rd.topicsdf["VotesPercentage"] = [prob for tup in rd.topicsdf.itertuples()
                                  if (topic := tup.Topic)
                                  if (votesum := rd.topicvotesum.loc[topic].Votes)
                                  if (prob := int(tup.Votes)/votesum)]

/var/folders/40/916hyvjs6cxgs7j_l2tp9bv00000gn/T/ipykernel_19025/1102643761.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rd.topicsdf["VotesPercentage"] = [prob for tup in rd.topicsdf.itertuples()


In [14]:
rd.topicsdf.head()

,Topic,StartVerseId,EndVerseId,Votes,PassageLength,UsableRange,UsableReference,VotesPercentage
0,10 commandments,02020001,02020026,291,26,True,True,0.093539
1,10 commandments,48005014,<NA>,140,1,False,True,0.045002
2,10 commandments,45013008,45013010,114,3,True,True,0.036644
3,10 commandments,05004013,<NA>,101,1,False,True,0.032465
4,10 commandments,02034028,<NA>,93,1,False,True,0.029894


In [17]:
# what's the distribution of VotesPercentage values?
rd.topicsdf.VotesPercentage.describe()

count    457465.000000
mean          0.014672
std           0.044672
min           0.000012
25%           0.001429
50%           0.005252
75%           0.011681
max           1.000000
Name: VotesPercentage, dtype: float64

In [19]:
# compare the top passages by VotesPercentage for "alcohol": 8 vs 131
alcoholdf = rd.topicsdf[rd.topicsdf.Topic == "alcohol"]
print(f"All rows: {len(alcoholdf)}")
alcoholdf[alcoholdf.VotesPercentage > 0.02]

All rows: 131


,Topic,StartVerseId,EndVerseId,Votes,PassageLength,UsableRange,UsableReference,VotesPercentage
8815,alcohol,49005018,<NA>,603,1,False,True,0.052001
8816,alcohol,20020001,<NA>,510,1,False,True,0.043981
8817,alcohol,54005023,<NA>,379,1,False,True,0.032684
8818,alcohol,20023029,20023035,326,7,True,True,0.028113
8819,alcohol,45014021,<NA>,310,1,False,True,0.026733
8820,alcohol,46006009,46006011,288,3,True,True,0.024836
8821,alcohol,23005011,<NA>,281,1,False,True,0.024232
8822,alcohol,23005022,<NA>,240,1,False,True,0.020697


In [ ]:
# write out top topics
toptopics.to_csv(DATAPATH / "OpenBible.info/toptopics.tsv", sep="\t")

In [23]:
# for each top topic, write out data for all verses
with (DATAPATH / "OpenBible.info/toptopicdata.tsv").open("w") as f:
    f.write("\t".join(["Index", "Topic", "StartVerseId", "EndVerseId", "Votes", "PassageLength", 
                       "UsableRange", "VotesPercentage"]) + "\n")
    for top in rd.toptopics.Topic:
        outdf = rd.topicsdf[rd.topicsdf.Topic == top].drop("UsableReference", axis="columns")
        outdf.to_csv(f, sep="\t", header=False)

In [24]:
# for each top topic, write out only rows whose VotesPercentage > 0.02
with (DATAPATH / "OpenBible.info/toptopictopversedata.tsv").open("w") as f:
    f.write("\t".join(["Index", "Topic", "StartVerseId", "EndVerseId", "Votes", "PassageLength", 
                       "UsableRange", "VotesPercentage"]) + "\n")
    for top in rd.toptopics.Topic:
        outdf = rd.topicsdf[rd.topicsdf.Topic == top].drop("UsableReference", axis="columns")
        # this figure is heuristic, based on the distribution of VotesPercentage values
        topversedf = outdf[outdf.VotesPercentage > 0.02]
        topversedf.to_csv(f, sep="\t", header=False)

In [21]:
from src import DATAPATH

In [ ]:
stop

## Expanding the Data: Topic Overlap

Topics overlap due to:
* typos (tatoo vs tattoo)
* general/specific 
* synonymy

Approach for similar strings:
* set a threshold S for similarity
* Compare all pairs T1, T2
* For all pairs whose similarity > S, combine them
* Repeat, averaging similarity across members of each set TS1, TS2. Might need to relax S given the averaging strategy?

Once term sets are combined, combine their verse inventories, averaging their votes. Potentially discard verses with counts below a threshold? 


In [ ]:
# verses for "helping others": 597 of them, so a much larger spread than some other topics
rd.display_topic_data("helping others")
# print(f"{len(df[df.Topic.str.startswith('helping others')])} verses with {df[df.Topic.str.startswith('helping others')].Votes.sum()} votes")

In [ ]:
helptopic = "helping others"
helpdf = rd.df[rd.df.Topic == helptopic]
helpdf

In [ ]:
helpvotesmedian = helpdf.Votes.median()
helpdf[helpdf.Votes > helpvotesmedian]

In [ ]:
helpabovemedian = helpdf[helpdf.Votes >= helpvotesmedian]
helpbelowmedian = helpdf[helpdf.Votes < helpvotesmedian]
print(f"Votes above median: {len(helpabovemedian)} verses, {helpabovemedian.Votes.sum()} votes ")
print(f"Votes below median: {len(helpbelowmedian)} verses, {helpbelowmedian.Votes.sum()} votes ")

## Multi-word Topics

How many topics have multiple terms, and what's the distribution?

In [ ]:
# get NLTK stopwords and make a superstring removing stop words
import nltk
from nltk.corpus import stopwords
english_stop_words = stopwords.words('english')
def slugify(string: str) -> str:
    """Remove stop words from string and join the results."""
    return "".join([s for s in string.split(" ") if s not in english_stop_words])


In [ ]:
topicsdf = pd.DataFrame(rd.df.Topic.value_counts())
topicsdf = topicsdf.reset_index()
topicsdf.columns = ["Topic", "Count"]
#topicsdf
topicsdf["AllTermsCount"] = topicsdf.apply(lambda t: len(t.Topic.split(" ")), axis=1)
# remove stopwords
topicsdf["SlugTerm"] = topicsdf.apply(lambda t: slugify(t.Topic), axis=1)
topicsdf.head()

In [ ]:
# make a big NxN matrix of all the SlugTerm values
from thefuzz import fuzz
slugterms = list(topicsdf.SlugTerm)
# Create a DataFrame with each string as a row, and all the strings as columns
slugdf = pd.DataFrame("" * len(slugterms), columns=slugterms, index=slugterms)

for i in range(len(slugterms)):
    st1 = slugterms[i]
    for j in range(i+1, len(slugterms)):
        st2 = slugterms[j]
        slugdf.loc[st1,st2] = fuzz.ratio(st1, st2)/100
slugdf.head()

In [ ]:
len(slugterms)

In [ ]:
# quite a few 2-3 word topics, and even one with 12 words! "were does the bible teach that jesus is the so..."
topicsdf.Termcount.value_counts()

In [ ]:
topicsdf[topicsdf.Termcount==8].Topic

In [ ]:
topicsdf[topicsdf.Termcount==12].Topic

In [ ]:
# distributional statistics for Votes
# Unlike the website display, it looks like only verses with at least 10 votes are included in the downloaded data
# The median number of votes is 30
# The standard deviation is very large! A lot of strong outliers at the upper end apparently. 
rd.df.Votes.describe()

In [ ]:
# focusing on the fat head
rd.df.Votes.describe(percentiles=[.75, .80, .85, .90, .95])

In [ ]:
# the topic+verse with the most votes: "helping others"
rd.df[rd.df.Votes == 24420]

In [ ]:
# other topics for the most popular verse. 
rd.df[rd.df.StartVerseId=="50002004"]

In [ ]:
# 50002005 is within the range for "affliction" above: does it also occur as a start or end?
# yes: a lot! This suggests we need to enumerate ranges into their components for better verse counting
# but just multiplying e.g. a range of five into five rows would seriously overweight their votes. 
# Dividing their votes by the range seems like it might _underweight_ their votes. 
# It's the old "how to score ranges" problem. 
rd.df[(rd.df.StartVerseId=="50002005") | (rd.df.EndVerseId=="50002005")]

In [ ]:
# comparing to verses for "tattoo*": 83 of them (
topicsubstr = "tattoo"
tattoodf = rd.df[rd.df.Topic.str.startswith(topicsubstr)]
print(f"{len(tattoodf)} verses with {tattoodf.Votes.sum()} votes")
tattoodf.Topic.value_counts()

In [ ]:
from clearlib.util import listalign
listalign.compare(["tattoos", "and", "piercings"], ["tattoo", "and", "body", "piercings"])